## Import libraries

In [13]:
import pandas as pd
import imaplib
import email
from email.header import decode_header
import re
from collections import Counter
from datetime import datetime

## Server imap data

In [8]:
IMAP_SERVER = 'imap.yandex.ru'    # imap.mail.ru for mail
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'

## Connect to mail

In [9]:
# connect
mail = imaplib.IMAP4_SSL(IMAP_SERVER)

In [10]:
# login
mail.login(EMAIL_ACCOUNT, PASSWORD)

('OK', [b'LOGIN Completed.'])

In [13]:
# select inbox
mail.select('inbox')

('OK', [b'3016'])

In [ ]:
mail.logout()

## Get numbor of inbox messages

In [35]:
IMAP_SERVER = 'imap.yandex.ru'    # imap.mail.ru for mail
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)
mail.select('inbox')

# Get number of letters
status, response = mail.search(None, 'ALL')    # 'UNSEEN', 'SEEN'
if status == 'OK':
    email_ids = response[0].split()
    num_emails = len(email_ids)
    print(f'Number of inbox letters: {num_emails}')

mail.logout()

Number of inbox letters: 3016


('BYE', [b'IMAP4rev1 Server logging out'])

## Get sender and subject of last 10 messages

In [21]:
mail.select('inbox')

status, response = mail.search(None, 'ALL')
if status == 'OK':
    email_ids = response[0].split()
    for email_id in email_ids[:-10:-1]:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (FROM SUBJECT)])')
        
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    from_ = msg.get('From')
                    subject = msg.get('Subject')

                    
                    # extract sender
                    email_address = re.search(r'<(.+?)>', from_)
                    if email_address:
                        from_ = email_address.group(1) 

                    #decode 
                    from_ = decode_header(from_)[0][0]
                    subject = decode_header(subject)[0][0]

                    
                    # convert to string
                    if isinstance(from_, bytes):
                        from_ = from_.decode()
                    if isinstance(subject, bytes):
                        subject = subject.decode()

                    print(f'From: {from_}')
                    print(f'Subject: {subject}')
                    print('-' * 50)

From: ic@lamoda.ru
Subject: Переход на новую систему оформления командировок КМР.Travel
--------------------------------------------------
From: viktoriya.barhatova@lamoda.ru
Subject: CALL SHEET 17/02 LAMODA ММ СТ М 11-14
--------------------------------------------------
From: pavel.ponomarenko@lamoda.ru
Subject: LAMODA CALLSHEET 17/02 СММ ВИДЕО Б ФЕВ
--------------------------------------------------
From: mariya.baranova@lamoda.ru
Subject: Re: CALL SHEET 14/02 ЭФИР LAMODA SPORT 25/02 ТЕСТ
--------------------------------------------------
From: info@calendar.yandex.ru
Subject: СММ ВИДЕО Б ФЕВ ПАША
--------------------------------------------------
From: varvaraandreeva@lamoda.ru
Subject: CALL SHEET 17/02 FOUNDS 
--------------------------------------------------
From: aleksandra.kazakova@lamoda.ru
Subject: Re: CALL SHEET 14/02 ЭФИР LAMODA SPORT 25/02 ТЕСТ
--------------------------------------------------
From: ic@lamoda.ru
Subject: КЭДО VK: что важно сделать до 17 февраля
---------

### Get the most frequently encountered authors

In [34]:
# Data to connect
IMAP_SERVER = 'imap.yandex.ru'    # imap.mail.ru for mail
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'

# Connection to imap server
mail = imaplib.IMAP4_SSL(IMAP_SERVER)

# log in
mail.login(EMAIL_ACCOUNT, PASSWORD)

# select inbox folder
mail.select('inbox')

# get all messages
status, response = mail.search(None, 'UNSEEN')
if status == 'OK':
    email_ids = response[0].split()

    # Create Counter for counting numbers of messages from each sender
    sender_counter = Counter()

    # get messages by ids
    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (FROM)])')
        if status == 'OK':
        # decode headers
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    from_ = msg.get('From')
                    email_address = re.search(r'<(.+?)>', from_)
                    if email_address:
                        from_ = email_address.group(1)

                    # add seneder to Counter
                    sender_counter[from_] += 1

    top_senders = sender_counter.most_common(5)
    print('Top most frequent senders: ')
    for sender, counter in top_senders:
        print(f'{sender}: {counter} messages')

mail.logout()
            

Top most frequent senders: 
internalcommunications@lamoda.ru: 361 messages
ekaterina.borovik@lamoda.ru: 96 messages
viktoriya.barhatova@lamoda.ru: 93 messages
ekaterina.volozhina@lamoda.ru: 92 messages
info@calendar.yandex.ru: 76 messages


('BYE', [b'IMAP4rev1 Server logging out'])

## Get subjects with 'CALLSHEET' in subject and with a date between 2024-10-01 and 2024-12-01

In [11]:
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'

SEARCH_WORD = 'SHEET'

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)

mail.select('inbox')

date_start = datetime.strptime('2024-10-01', '%Y-%m-%d').strftime('%d-%b-%Y')  # 01-Oct-2024
date_end = datetime.strptime('2024-12-01', '%Y-%m-%d').strftime('%d-%b-%Y')   # 01-Dec-2024

status, response = mail.search(None, f'(SINCE "{date_start}" BEFORE "{date_end}")')
if status == 'OK':
    email_ids = response[0].split()

    for email_id in email_ids[::-1]:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (SUBJECT)])')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1]).get('Subject')
                    if msg:
                        subject, encoding = decode_header(msg)[0]
                        
                    # if subject is encoded in base64 or other
                        if isinstance(subject, bytes):
                            subject = subject.decode(encoding or 'utf-8')
                        date = re.search(r'(\d{2}/\d{2})', subject)
                        # if date:
                        #     date = datetime.strptime(date[1], "%d/%m").date().replace(year = 2025)
                        if subject and 'SHEET' in subject and 'СММ' in subject:
                            print(f'Subject contains word {SEARCH_WORD}: {subject}')

mail.logout()   

Subject contains word SHEET: Re: CALL SHEET 19/11 LAMODA СММ ПРЕМ НОЯБ
Subject contains word SHEET: CALL SHEET 19/11 LAMODA СММ ПРЕМ НОЯБ
Subject contains word SHEET: CALL SHEET 14/11 СММ ВИДЕО Б НОЯБ
Subject contains word SHEET: Re: CALL SHEET 13/11 LAMODA СММ М НОЯБ
Subject contains word SHEET: Re: CALL SHEET 13/11 LAMODA СММ М НОЯБ
Subject contains word SHEET: CALL SHEET 13/11 LAMODA СММ М НОЯБ 
Subject contains word SHEET: CALL SHEET 11/11 СММ ВИДЕО НОЯБ 
Subject contains word SHEET: Re: CALL SHEET 07/11 Lamoda СММ + ОБТР Ж НОЯБ
Subject contains word SHEET: CALL SHEET 07/11 Lamoda СММ + ОБТР Ж НОЯБ
Subject contains word SHEET: Re: CALL SHEET  30/10 Lamoda СММ ВИДЕО Б ОКТ
Subject contains word SHEET: CALL SHEET  30/10 Lamoda СММ ВИДЕО Б ОКТ
Subject contains word SHEET: Re: LAMODA CALLSHEET 15/10 СММ ПРЕМ ОКТ
Subject contains word SHEET: LAMODA CALLSHEET 15/10 СММ ПРЕМ ОКТ
Subject contains word SHEET: CALL SHEET Lamoda 09/10 СММ ВИДЕО ОКТ
Subject contains word SHEET: Re: LAMODA CALLS

('BYE', [b'IMAP4rev1 Server logging out'])

In [13]:
# Get list of search data

IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'

SEARCH_WORD = 'SHEET'
SEARCH_LIST = []

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)

mail.select('inbox')

date_start = datetime.strptime('2024-10-01', '%Y-%m-%d').strftime('%d-%b-%Y')  # 01-Oct-2024
date_end = datetime.strptime('2024-12-01', '%Y-%m-%d').strftime('%d-%b-%Y')   # 01-Dec-2024

status, response = mail.search(None, f'(SINCE "{date_start}" BEFORE "{date_end}")')
if status == 'OK':
    email_ids = response[0].split()

    for email_id in email_ids[::-1]:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (SUBJECT)])')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1]).get('Subject')
                    if msg:
                        subject, encoding = decode_header(msg)[0]
                        
                    # if subject is encoded in base64 or other
                        if isinstance(subject, bytes):
                            subject = subject.decode(encoding or 'utf-8')
                        if subject and 'SHEET' in subject and 'СММ' in subject:
                            SEARCH_LIST.append(f'Subject contains word {SEARCH_WORD}: {subject}')

mail.logout()   

('BYE', [b'IMAP4rev1 Server logging out'])

## Let's send data from one email to another

In [2]:
import imaplib
import email
from email.header import decode_header
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [15]:
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'
SMTP_SERVER = 'smtp.yandex.ru'
SMTP_PORT = 587

# Create message object
msg = MIMEMultipart()
msg['FROM'] = EMAIL_ACCOUNT
msg['TO'] = 'konsin1988@gmail.com'
msg['Subject'] = 'Messages with SMM content'

# Body 
body = '\n'.join(SEARCH_LIST)
msg.attach(MIMEText(body, 'plain'))

# Connect to smtp server 
server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
server.starttls()
server.login(EMAIL_ACCOUNT, PASSWORD)

# Send message
server.sendmail(EMAIL_ACCOUNT, 'konsin1988@gmail.com', msg.as_string())

# Quit server
server.quit()
print('Message sended successful')

Message sended successful


## Let's make emails sent from certain addresses unread

In [4]:
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'
DATE_START = datetime.strptime('2025-02-01', '%Y-%m-%d').strftime('%d-%b-%Y')  # 01-Oct-2024
DATE_END = datetime.strptime('2025-02-12', '%Y-%m-%d').strftime('%d-%b-%Y') 

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)

mail.select('inbox')

status, response = mail.search(None, f'SINCE "{DATE_START}" BEFORE "{DATE_END}"')
if status == 'OK':
    email_ids = response[0].split()
    for email_id in email_ids[::-1]:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (FROM)])')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    from_ = msg.get('From')

                    email_address = re.search(r'<(.+?)>', from_)
                    if email_address:
                        from_ = email_address[1]

                    # delete flag SEEN from messages with 'internalcommunications' in email_address
                    if from_ and ('internalcommunications' in from_ or 'calendar' in from_):
                        mail.store(email_id, '-FLAGS', '\\SEEN')
                        print(f'Email {email_id.decode()} marked as unread')
    

mail.logout()

Email 2991 marked as unread
Email 2990 marked as unread
Email 2989 marked as unread
Email 2987 marked as unread
Email 2984 marked as unread
Email 2983 marked as unread
Email 2982 marked as unread
Email 2973 marked as unread
Email 2970 marked as unread
Email 2969 marked as unread
Email 2968 marked as unread
Email 2962 marked as unread
Email 2961 marked as unread
Email 2960 marked as unread
Email 2956 marked as unread
Email 2954 marked as unread
Email 2953 marked as unread
Email 2952 marked as unread
Email 2951 marked as unread
Email 2946 marked as unread
Email 2945 marked as unread
Email 2944 marked as unread


('BYE', [b'IMAP4rev1 Server logging out'])

## Remove emails from a specific sender

In [5]:
import imaplib
import email
from email.header import decode_header
from datetime import datetime

In [12]:
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'dmitry.konshin@lamoda.ru'
PASSWORD = 'solsvcfuyqxmqigb'
DATE_START = datetime.strptime('2025-01-01', '%Y-%m-%d').strftime('%d-%b-%Y')
DATE_END = datetime.strptime('2025-02-14', '%Y-%m-%d').strftime('%d-%b-%Y')
SENDER_TO_REMOVE = 'Denis Varankin'

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)
mail.select('inbox')

status, response = mail.search(None, f'(SINCE "{DATE_START}" BEFORE "{DATE_END}")')
if status == 'OK':
    email_ids = response[0].split()

    for email_id in email_ids:
        status, msg_data = mail.fetch(email_id, '(BODY[HEADER.FIELDS (FROM)])')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    from_ = msg.get('From')
                    from_ = decode_header(from_)[0][0]
                    if isinstance(from_, bytes):
                        from_ = from_.decode()
                    if from_ and SENDER_TO_REMOVE in from_:
                        mail.store(email_id, '+FLAGS', '\\Deleted')
                        print(f'Email {email_id.decode()} marked as deleted')

    mail.expunge()
mail.logout()

Email 2848 marked as deleted
Email 2877 marked as deleted
Email 2886 marked as deleted
Email 2898 marked as deleted
Email 2924 marked as deleted
Email 2960 marked as deleted
Email 2980 marked as deleted
Email 2982 marked as deleted
Email 2993 marked as deleted


('BYE', [b'IMAP4rev1 Server logging out'])

## Open the file received by mail and upload it to clickhouse

In [15]:
import pandas as pd
import imaplib
import email
from email.header import decode_header
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [16]:
# imap 
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'konsin1988@yandex.ru'
PASSWORD = 'vqmobuumbfxufahi'
SMTP_SERVER = 'smtp.yandex.ru'
SMTP_PORT = 587
TO_EMAIL = 'konsin1988@yandex.ru'

In [17]:
# create message
msg = MIMEMultipart()
msg['From'] = EMAIL_ACCOUNT
msg['To'] = TO_EMAIL
msg['Subject'] = 'data_20240714721i33'

body = 'Attached is a data file. Have a good day!'
msg.attach(MIMEText(body, 'plain'))

# attach file
filename = 'data_20240714721i33.xml'
with open(filename, 'rb') as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header(
        'Content-Disposition',
        f'attachment; filename={filename}'
    )
    msg.attach(part)

# Send message
server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
server.starttls()
server.login(EMAIL_ACCOUNT, PASSWORD)
server.send_message(msg)
server.quit()

print(f'Message with {filename} sended successful')

Message with data_20240714721i33.xml sended successful


## Get sender, subject, date and time from message with attachment. Loading attached file.

In [18]:
import imaplib
import email
from email.header import decode_header

In [21]:
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'konsin1988@yandex.ru'
PASSWORD = 'vqmobuumbfxufahi'
SEARCH_INFO = 'data_20240714721i33'

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, PASSWORD)
mail.select('inbox')

status, response = mail.search(None, 'UNSEEN')
if status == 'OK':
    for email_id in response[0].split():
        status, msg_data = mail.fetch(email_id, '(RFC822)')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    date = msg.get('Date')
                    subject = msg.get('Subject')
                    sender = msg.get('From')

                    if subject:
                        subject_decoded, encoding = decode_header(subject)[0]
                        if isinstance(subject_decoded, bytes):
                            subject_decoded = subject_decoded.decode(encoding or 'utf-8')

                        if SEARCH_INFO in subject_decoded.lower():
                            print(f'Date: {date}')
                            print(f'Sender: {sender}')
                            print(f'Subject: {subject_decoded}')

                            # check attaches
                            for part in msg.walk():
                                if part.get_content_disposition() == 'attachment':
                                    filename = part.get_filename()
                                    # save file
                                    if filename:
                                        with open(filename, 'wb') as file:
                                            file.write(part.get_payload(decode = True))
                                        print(f'Attachment save to {filename}')

mail.logout()

Date: Sat, 15 Feb 2025 10:18:56 +0300
Sender: konsin1988@yandex.ru
Subject: data_20240714721i33
Attachment save to data_20240714721i33.xml


('BYE', [b'IMAP4rev1 Server logging out'])

## Loading data from attached email file to clickhouse

In [33]:
import pandas as pd
import clickhouse_connect
import imaplib
import email
from email.header import decode_header
import io

In [84]:
# mail
IMAP_SERVER = 'imap.yandex.ru'
EMAIL_ACCOUNT = 'konsin1988@yandex.ru'
EMAIL_PASS = 'vqmobuumbfxufahi'

# file
DATA_FILE = 'data_'

#clickhouse
CLICKHOUSE_USER = 'konsin1988'
CLICKHOUSE_PASS = 'konsin1988konsin1988'
CLICKHOUSE_HOST = 'localhost'
CLICKHOUSE_PORT = 8123
CLICKHOUSE_DB = 'mail_data'
CLICKHOUSE_TABLE = 'loans'

In [3]:
client = clickhouse_connect.get_client(host=CLICKHOUSE_HOST, port=CLICKHOUSE_PORT, username=CLICKHOUSE_USER, password=CLICKHOUSE_PASS)

In [6]:
# Create role admin IF NOT EXISTS
query = r'''
    CREATE ROLE IF NOT EXISTS admin
'''
client.command(query)

In [7]:
# Grant to admin full access 
query = r'''
    GRANT ALL ON *.* TO admin WITH GRANT OPTION
'''
client.command(query)

In [5]:
query = r'''
    CREATE DATABASE IF NOT EXISTS mail_data;
'''
client.command(query)

In [88]:
query = r'''
    DROP TABLE IF EXISTS mail_data.loans
'''
client.command(query)

query = r'''
    CREATE TABLE IF NOT EXISTS mail_data.loans(
        age Int64,
        sex Nullable(String),
        job Nullable(Int64),
        housing Nullable(String),
        saving_accounts Nullable(String),
        checking_account Nullable(String),
        credit_amount Int64,
        duration Int64,
        purpose Nullable(String),
        default Int64,
        contract_dt DateTime('UTC'),
        client_id Int64
    ) ENGINE MergeTree PARTITION BY toYYYYMM(contract_dt) ORDER BY contract_dt SETTINGS index_granularity = 256;
'''
client.command(query)

In [90]:
query = r'''
    SELECT *
    FROM mail_data.loans
    LIMIT 5
'''
pd.DataFrame(client.query(query).named_results())

""


In [91]:
# make mail with attached data unread
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, EMAIL_PASS)

mail.select('inbox')

status, response = mail.search(None, 'SEEN')
if status == 'OK':
    email_ids = response[0].split()
    for email_id in email_ids[::-1]:
        status, msg_data = mail.fetch(email_id, '(RFC822)')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    subject = msg.get('Subject')
                    
                    if subject:
                        subject_decoded, encoding = decode_header(subject)[0]
                        if isinstance(subject_decoded, bytes):
                            subject_decoded = subject_decoded.decode(encoding or 'utf-8')
                    else:
                        subject_decoded = ''

                    # delete flag SEEN from messages with 'internalcommunications' in email_address
                    if SEARCH_INFO in subject.lower():
                        mail.store(email_id, '-FLAGS', '\\SEEN')
                        print(f'Email {email_id.decode()} marked as unread')
    

mail.logout()

Email 4 marked as unread


('BYE', [b'IMAP4rev1 Server logging out'])

In [92]:
# get file from email and insert xml data to clickhouse
mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, EMAIL_PASS)

mail.select('inbox')

content = None
status, response = mail.search(None, 'UNSEEN')
if status == 'OK':
    for email_id in response[0].split():
        status, msg_data = mail.fetch(email_id, '(RFC822)')
        if status == 'OK':
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])

                    date = msg.get('Date')
                    subject = msg.get('Subject')

                    if subject:
                        subject_decoded, encoding = decode_header(subject)[0]
                        if isinstance(subject_decoded, bytes):
                            subject_decoded = subject_decoded.decode(encoding or 'utf-8')
                    else:
                        subject_decoded = ''

                    for part in msg.walk():
                        if part.get_content_disposition() == 'attachment':
                            attachment_name = part.get_filename() or ''
                            # print(part.get_content_type())
                            if DATA_FILE in attachment_name and part.get_content_type() == 'application/octet-stream':
                                content = part.get_payload(decode = True).decode('utf-8')
                                print(f'The file attached to the email id_{email_id.decode()} was recorded in the database')
    
    if content:
        client.insert_df('mail_data.loans', pd.read_xml(io.StringIO(content), dtype={
            'contract_dt':'datetime64[ns]'    
        }
        ))
        

    mail.expunge()
mail.logout()

The file attached to the email id_4 was recorded in the database


('BYE', [b'IMAP4rev1 Server logging out'])

In [95]:
query = r'''
    SELECT COUNT(*) AS `Number of rows`
    FROM mail_data.loans
'''
pd.DataFrame(client.query(query).named_results())

,Number of rows
0,1000
